# Exercice — Gouvernance & Catalogage (GreenFarm)

Objectif : transformer un CSV en tables Delta **gouvernées** via Unity Catalog :
- `agriculture.raw.sensors` (raw zone)
- `agriculture.analytics.sensors_daily` (analytics agrégée)
Puis gérer les accès sur le schéma analytics.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

csv_path = "/Volumes/workspace/default/oc-lakehouse/greenfarm_sensors.csv"

catalog = "agriculture"
raw_schema = "raw"
analytics_schema = "analytics"

raw_table = f"{catalog}.{raw_schema}.sensors"
analytics_table = f"{catalog}.{analytics_schema}.sensors_daily"

print("CSV:", csv_path)
print("Raw table:", raw_table)
print("Analytics table:", analytics_table)


CSV: /Volumes/workspace/default/oc-lakehouse/greenfarm_sensors.csv
Raw table: agriculture.raw.sensors
Analytics table: agriculture.analytics.sensors_daily


## SQL — Créer la structure Lakehouse (catalog + schémas)

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS agriculture;

CREATE SCHEMA IF NOT EXISTS agriculture.raw;
CREATE SCHEMA IF NOT EXISTS agriculture.analytics;

## Charger le CSV + écrire en Delta (RAW)

In [0]:
schema = T.StructType([
    T.StructField("measurement_ts", T.StringType(), True),
    T.StructField("parcel_id", T.StringType(), True),
    T.StructField("sensor_id", T.StringType(), True),
    T.StructField("humidity", T.DoubleType(), True),
    T.StructField("temperature", T.DoubleType(), True),
    T.StructField("soil_ph", T.DoubleType(), True),
    T.StructField("battery_pct", T.DoubleType(), True),
])

df_raw = (
    spark.read
    .option("header", True)
    .schema(schema)
    .csv(csv_path)
    .withColumn("measurement_ts", F.to_timestamp("measurement_ts"))
)

display(df_raw.limit(10))
print("Rows:", df_raw.count())

# Table managée Unity Catalog (saveAsTable)
spark.sql(f"DROP TABLE IF EXISTS {raw_table}")

(df_raw.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable(raw_table)
)

print("✅ Table RAW créée:", raw_table)

measurement_ts,parcel_id,sensor_id,humidity,temperature,soil_ph,battery_pct
2026-01-01T09:21:00.000Z,West-1,S-013,57.7,9.6,5.69,24.2
2026-01-01T10:10:00.000Z,West-1,S-006,37.7,22.5,6.88,28.0
2026-01-01T10:29:00.000Z,South-1,S-003,61.2,20.0,5.7,52.0
2026-01-01T10:33:00.000Z,North-1,S-033,63.8,11.8,6.43,47.4
2026-01-01T10:40:00.000Z,North-1,S-012,44.8,31.1,7.12,78.6
2026-01-01T10:44:00.000Z,North-1,S-036,76.1,29.0,6.61,55.7
2026-01-01T11:10:00.000Z,East-2,S-022,79.1,34.5,7.4,43.5
2026-01-01T15:06:00.000Z,North-2,S-025,59.6,22.5,7.6,42.4
2026-01-01T15:36:00.000Z,North-2,S-023,46.7,31.5,6.91,65.9
2026-01-01T16:07:00.000Z,South-1,S-037,56.2,32.4,6.27,50.9


Rows: 2000
✅ Table RAW créée: agriculture.raw.sensors


## SQL — Vérifier accès et table managée (et voir le chemin)

In [0]:
%sql
-- Vérifier l'accès
SELECT * FROM agriculture.raw.sensors LIMIT 10;

-- Vérifier la table (managed/external) et récupérer l'info de location
DESCRIBE EXTENDED agriculture.raw.sensors;

-- En UC, DESCRIBE DETAIL est souvent le plus lisible
DESCRIBE DETAIL agriculture.raw.sensors;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,b570525d-e339-48a0-a2b3-b22205fb0722,agriculture.raw.sensors,null,,2026-01-23T10:23:12.985Z,2026-01-23T10:23:15.000Z,List(),List(),1,27301,"Map(delta.parquet.compression.codec -> zstd, delta.enableDeletionVectors -> true)",3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


## Créer la table analytics (agrégée par jour & parcelle)

On sélectionne les colonnes utiles et on agrège :
- jour (date sans heure)
- parcelle
- avg_humidity, avg_temperature

In [0]:
df_daily = (
    spark.table(raw_table)
    .select(
        F.to_date("measurement_ts").alias("day"),
        F.col("parcel_id"),
        F.col("humidity"),
        F.col("temperature"),
    )
    .groupBy("day", "parcel_id")
    .agg(
        F.avg("humidity").alias("avg_humidity"),
        F.avg("temperature").alias("avg_temperature"),
    )
)

display(df_daily.orderBy("day", "parcel_id").limit(20))

spark.sql(f"DROP TABLE IF EXISTS {analytics_table}")

(df_daily.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable(analytics_table)
)

print("✅ Table analytics créée:", analytics_table)

day,parcel_id,avg_humidity,avg_temperature
2026-01-01,East-1,23.1,25.4
2026-01-01,East-2,51.27499999999999,21.150000000000002
2026-01-01,North-1,59.3,25.05
2026-01-01,North-2,53.150000000000006,27.0
2026-01-01,South-1,58.6,19.6
2026-01-01,South-2,44.3,17.674999999999997
2026-01-01,West-1,40.56666666666667,21.866666666666664
2026-01-02,East-1,59.199999999999996,23.933333333333334
2026-01-02,East-2,45.63333333333333,16.95
2026-01-02,North-1,49.225,10.5


✅ Table analytics créée: agriculture.analytics.sensors_daily


## SQL — Interroger les deux tables via le catalogue

In [0]:
%sql
SELECT * FROM agriculture.raw.sensors LIMIT 10;

SELECT * FROM agriculture.analytics.sensors_daily
ORDER BY day, parcel_id
LIMIT 20;

day,parcel_id,avg_humidity,avg_temperature
2026-01-01,East-1,23.1,25.4
2026-01-01,East-2,51.27499999999999,21.150000000000002
2026-01-01,North-1,59.3,25.05
2026-01-01,North-2,53.150000000000006,27.0
2026-01-01,South-1,58.6,19.6
2026-01-01,South-2,44.3,17.674999999999997
2026-01-01,West-1,40.56666666666667,21.866666666666664
2026-01-02,East-1,59.199999999999996,23.933333333333334
2026-01-02,East-2,45.63333333333333,16.95
2026-01-02,North-1,49.225,10.5


## SQL — Gérer les accès (lecture pour tous sur analytics)

In [0]:
%sql
-- Autoriser la lecture sur toutes les tables (présentes et futures)
GRANT SELECT ON SCHEMA agriculture.analytics
TO `alexandre.bergere@datalex.io`;

SHOW GRANTS ON SCHEMA agriculture.analytics;

Principal,ActionType,ObjectType,ObjectKey
alexandre.bergere@datalex.io,SELECT,SCHEMA,agriculture.analytics
